In [12]:
import pandas as pd 
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Input
from keras.layers import Dropout
from keras.callbacks import EarlyStopping
from keras.applications.inception_v3 import InceptionV3
from keras.applications.mobilenet import MobileNet
from keras.layers import merge, Conv2D, MaxPooling2D, GlobalAveragePooling1D, Flatten, GlobalAveragePooling2D, ZeroPadding2D
from keras.models import load_model, Model
from keras.layers.normalization import BatchNormalization
from keras.callbacks import History ,ModelCheckpoint
from keras.layers import Activation, LeakyReLU, GlobalMaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.models import load_model
import collections
import sys
import pickle
import os

import matplotlib.pyplot as plt
import skimage
import skimage.io
from sklearn import preprocessing
from squeezenet import SqueezeNet

%matplotlib inline
%env CUDA_VISIBLE_DEVICES=4

env: CUDA_VISIBLE_DEVICES=4


In [2]:
train_X = np.load("../features/train_X.npy")/255
valid_X = np.load("../features/valid_X.npy")/255
test_X = np.load("../features/test_X.npy")/255

In [3]:
# prepare dataset
# train_img_folder = "../dlcv_final_2_dataset/train/"
# train_img_path = sorted(os.listdir(train_img_folder))
# train_X_img = np.array([skimage.io.imread(os.path.join(train_img_folder, path))
#               for path in train_img_path])/255.

with open("../dlcv_final_2_dataset/train_id.txt","r") as f:
    train_y = f.readlines()
    train_y = np.array([line.strip().split(" ")[1] for line in train_y])
    
# valid_img_folder = "../dlcv_final_2_dataset/val/"
# valid_img_path = sorted(os.listdir(valid_img_folder))
# valid_X_img = np.array([skimage.io.imread(os.path.join(valid_img_folder, path))
#               for path in valid_img_path])/255.

with open("../dlcv_final_2_dataset/val_id.txt","r") as f:
    valid_y = f.readlines()
    valid_y = np.array([line.strip().split(" ")[1] for line in valid_y])

    
# test_img_folder = "../dlcv_final_2_dataset/test/"
# test_img_path = sorted(os.listdir(test_img_folder))
# test_X_img = np.array([skimage.io.imread(os.path.join(test_img_folder, path))
#               for path in test_img_path])/255.

In [4]:
# encoder = preprocessing.LabelEncoder()
# train_y_encoded = encoder.fit_transform(train_y)
# valid_y_encoded = encoder.fit_transform(valid_y)

lb = preprocessing.LabelBinarizer()
train_y_onehot = lb.fit_transform(train_y)
valid_y_onehot = lb.transform(valid_y)

In [5]:
train_aug = np.array([np.fliplr(img) for img in train_X])
train_y_onehot = np.vstack((train_y_onehot,train_y_onehot))

train_X_aug = np.vstack((train_X,train_aug))
print(train_X_aug.shape)

In [ ]:
datagen = ImageDataGenerator(
            rotation_range=40,
            zoom_range=[0.85, 1.15],
            horizontal_flip=True)

In [8]:
train_X_aug[1].shape

(120, 90, 3)

In [9]:
batch_size = 64
epochs = 200

model = SqueezeNet(nb_classes=2360,input_shape=(120,90,3))

/home/thtang/DLCV2018SPRING/final/train/squeezenet.py:36: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(96, (7, 7), strides=(2, 2), name="conv1", kernel_initializer="he_normal", activation="relu")`
  conv1 = Convolution2D(96, 7, 7, activation='relu', subsample=(2, 2), init='he_normal', name='conv1')(input_image)
/home/thtang/DLCV2018SPRING/final/train/squeezenet.py:15: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (1, 1), name="fire2_squeeze", kernel_initializer="he_normal", activation="relu")`
  squeeze = Convolution2D(s_1x1, 1, 1, activation='relu', init='he_normal', name=name+'_squeeze')(x)
/home/thtang/DLCV2018SPRING/final/train/squeezenet.py:19: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (1, 1), name="fire2_expand_1x1", padding="same", kernel_initializer="he_normal", activation="relu")`
  expand_1x1 = Convolution2D(e_1x1, 1, 1, border_mode='same', activation='relu', init='he_normal', name=name+'_expand_1x1')

/home/thtang/DLCV2018SPRING/final/train/squeezenet.py:73: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=[<tf.Tenso...)`
  model = Model(input=input_image, output=[softmax])


In [10]:
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 120, 90, 3)   0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 57, 42, 96)   14208       input_1[0][0]                    
__________________________________________________________________________________________________
maxpool1 (MaxPooling2D)         (None, 28, 20, 96)   0           conv1[0][0]                      
__________________________________________________________________________________________________
fire2_squeeze (Conv2D)          (None, 28, 20, 16)   1552        maxpool1[0][0]                   
__________________________________________________________________________________________________
fire2_sque

In [11]:
optimizer = Adam(lr=1e-4,beta_1=0.5)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
hist = History()
save_model = ModelCheckpoint("../models/weights.{epoch:02d}-{val_acc:.2f}.hdf5", 
                             monitor='val_acc', save_best_only=True
                             , mode='auto', period=1)
# model.fit_generator(
#             datagen.flow(train_X_img, train_y_onehot, batch_size=batch_size), 
#             steps_per_epoch=10*len(train_X_img)//batch_size,
#             validation_data=(valid_X_img, valid_y_onehot),
#             epochs=epochs, callbacks=[hist], workers = 20 )

model.fit(train_X_aug, train_y_onehot, batch_size=batch_size,
            validation_data=(valid_X, valid_y_onehot),
            epochs=epochs, callbacks=[hist,save_model])

Train on 112950 samples, validate on 7211 samples
Epoch 1/200
112950/112950 [==============================] - 91s 805us/step - loss: 7.7061 - acc: 8.4993e-04 - val_loss: 7.5905 - val_acc: 0.0019
Epoch 2/200
112950/112950 [==============================] - 87s 773us/step - loss: 7.4166 - acc: 0.0048 - val_loss: 7.3373 - val_acc: 0.0079
Epoch 3/200
112950/112950 [==============================] - 87s 773us/step - loss: 7.0173 - acc: 0.0158 - val_loss: 6.8888 - val_acc: 0.0184
Epoch 4/200
112950/112950 [==============================] - 87s 774us/step - loss: 6.5044 - acc: 0.0366 - val_loss: 6.4039 - val_acc: 0.0429
Epoch 5/200
112950/112950 [==============================] - 87s 773us/step - loss: 6.0182 - acc: 0.0688 - val_loss: 5.8945 - val_acc: 0.0808
Epoch 6/200
112950/112950 [==============================] - 87s 773us/step - loss: 5.5793 - acc: 0.1032 - val_loss: 5.5220 - val_acc: 0.1119
Epoch 7/200
112950/112950 [==============================] - 87s 773us/step - loss: 5.2070 - a

112950/112950 [==============================] - 87s 771us/step - loss: 0.4424 - acc: 0.9516 - val_loss: 3.5792 - val_acc: 0.54832s - lo
Epoch 58/200
112950/112950 [==============================] - 87s 771us/step - loss: 0.4285 - acc: 0.9533 - val_loss: 3.3972 - val_acc: 0.5689
Epoch 59/200
112950/112950 [==============================] - 87s 773us/step - loss: 0.4113 - acc: 0.9551 - val_loss: 3.4281 - val_acc: 0.5648
Epoch 60/200
112950/112950 [==============================] - 87s 771us/step - loss: 0.4014 - acc: 0.9561 - val_loss: 3.5004 - val_acc: 0.5579
Epoch 61/200
112950/112950 [==============================] - 87s 772us/step - loss: 0.3895 - acc: 0.9573 - val_loss: 3.5650 - val_acc: 0.5505
Epoch 62/200
112950/112950 [==============================] - 87s 773us/step - loss: 0.3748 - acc: 0.9593 - val_loss: 3.3726 - val_acc: 0.5705
Epoch 63/200
112950/112950 [==============================] - 87s 772us/step - loss: 0.3651 - acc: 0.9602 - val_loss: 3.4104 - val_acc: 0.5650
Epoch

112950/112950 [==============================] - 75s 668us/step - loss: 0.1436 - acc: 0.9820 - val_loss: 3.3237 - val_acc: 0.5726
Epoch 115/200
112950/112950 [==============================] - 76s 670us/step - loss: 0.1414 - acc: 0.9820 - val_loss: 4.1140 - val_acc: 0.4830
Epoch 116/200
112950/112950 [==============================] - 76s 673us/step - loss: 0.1409 - acc: 0.9820 - val_loss: 3.4428 - val_acc: 0.5652
Epoch 117/200
112950/112950 [==============================] - 75s 665us/step - loss: 0.1380 - acc: 0.9825 - val_loss: 4.0795 - val_acc: 0.4888
Epoch 118/200
112950/112950 [==============================] - 76s 670us/step - loss: 0.1381 - acc: 0.9822 - val_loss: 3.2746 - val_acc: 0.5834
Epoch 119/200
112950/112950 [==============================] - 75s 665us/step - loss: 0.1371 - acc: 0.9822 - val_loss: 3.2880 - val_acc: 0.5797
Epoch 120/200
112950/112950 [==============================] - 75s 668us/step - loss: 0.1349 - acc: 0.9827 - val_loss: 3.3862 - val_acc: 0.5737
Epoch 

Epoch 171/200
112950/112950 [==============================] - 76s 669us/step - loss: 0.0897 - acc: 0.9874 - val_loss: 3.3830 - val_acc: 0.5659
Epoch 172/200
112950/112950 [==============================] - 76s 672us/step - loss: 0.0891 - acc: 0.9874 - val_loss: 3.2323 - val_acc: 0.5815
Epoch 173/200
112950/112950 [==============================] - 75s 665us/step - loss: 0.0881 - acc: 0.9876 - val_loss: 3.4991 - val_acc: 0.5521
Epoch 174/200
112950/112950 [==============================] - 76s 669us/step - loss: 0.0870 - acc: 0.9879 - val_loss: 3.3563 - val_acc: 0.5712
Epoch 175/200
112950/112950 [==============================] - 75s 668us/step - loss: 0.0859 - acc: 0.9881 - val_loss: 3.2370 - val_acc: 0.5795
Epoch 176/200
112950/112950 [==============================] - 75s 667us/step - loss: 0.0859 - acc: 0.9880 - val_loss: 3.4365 - val_acc: 0.5591
Epoch 177/200
112950/112950 [==============================] - 76s 671us/step - loss: 0.0843 - acc: 0.9881 - val_loss: 3.3127 - val_acc:

In [ ]:
model.summary()

In [ ]:
plt.plot(hist.history["acc"])
plt.plot(hist.history["val_acc"])
plt.show()